In [1]:
from constants.abs_paths import AbsDirPath
import pandas as pd

from processing_pipeline.utilities.data_transformation import split_dataset_by_repo_and_source, load_all_files

In [2]:
df = load_all_files(AbsDirPath.O_KEYWORDS_MATCHING)

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.issue.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.issue_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.release.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\google.deepvariant.v1.6.1.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\google.deepvariant.v1.6.1.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\

In [3]:
df.head()

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,nunique_matched_words,nsimilar
0,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,Given something that might be a URL (or might ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,1
1,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Given a URL, look for the corresponding datase...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,1
2,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Download to temporary file, then copy to cache...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,2
3,allenai/scispacy/v0.5.5.code_comment.availabil...,avail,avail,A candidate generator for entity linking to a ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,available,"available, Available, AVAILABLE",3,1
4,allenai/scispacy/v0.5.5.code_comment.availabil...,consist,consist,A spacy pipeline component which identifies en...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,consists,"consists, Consists",2,1


In [4]:
df.shape

(1601472, 12)

## Extract columns required for the next stage's prompt

In [17]:
required_columns_for_the_next_stage_prompt = ["sentence"]
default_columns = ["repo_id", "source"]
total_columns = required_columns_for_the_next_stage_prompt + default_columns

In [6]:
df2 = df.drop_duplicates(required_columns_for_the_next_stage_prompt)[total_columns]

In [7]:
df2.head()

,sentence,repo_id,source
0,Given something that might be a URL (or might ...,allenai/scispacy/v0.5.5,code_comment
1,"Given a URL, look for the corresponding datase...",allenai/scispacy/v0.5.5,code_comment
2,"Download to temporary file, then copy to cache...",allenai/scispacy/v0.5.5,code_comment
3,A candidate generator for entity linking to a ...,allenai/scispacy/v0.5.5,code_comment
4,A spacy pipeline component which identifies en...,allenai/scispacy/v0.5.5,code_comment


In [8]:
df2.shape

(561064, 3)

## Drop sentences that contain 5 or fewer words

In [9]:
df2['nwords'] = df2.sentence.str.count(" ") + 1
df2.sort_values(["nwords"], ascending=True).head(100)

,sentence,repo_id,source,nwords
214,Buffer,root-project/root/v6-32-06,code_comment,1
886,"console.log(""handleOverlayMouseMove"");",root-project/root/v6-32-06,code_comment,1
766,"([bd,An,Xn.W,SCALE],od)",root-project/root/v6-32-06,code_comment,1
3641,HandleCrossing(event);,root-project/root/v6-32-06,code_comment,1
600,Variable,root-project/root/v6-32-06,code_comment,1
...,...,...,...,...
4972,enum-specifier,root-project/root/v6-32-06,code_comment,1
2777,LoadedOffset,root-project/root/v6-32-06,code_comment,1
1274,symbol@got@tprel@ha,root-project/root/v6-32-06,code_comment,1
1273,RecoverableErrorHandler,root-project/root/v6-32-06,code_comment,1


In [10]:
max_nwords = df2.nwords.max()
max_nwords

np.int64(641)

In [11]:
max_nwords // 10

np.int64(64)

In [12]:
import plotly.express as px

# df2.hist(column="nwords", nbin)
px.histogram(df2, x="nwords", nbins=int(max_nwords * 2 * 2 // 10))


In [13]:
df2[(df2.nwords > 5) & (df2.nwords < 11)].sort_values("nwords", ascending=True)

,sentence,repo_id,source,nwords
10832,Global variables are always in scope.,root-project/root/v6-32-06,code_comment,6
10821,Operand 1 is implicitly zero extended.,root-project/root/v6-32-06,code_comment,6
10789,Check for GCC complex integer extension.,root-project/root/v6-32-06,code_comment,6
10736,! Obsolete decompression functions (since v1.8.0),root-project/root/v6-32-06,code_comment,6
10699,< prior pdf for nuisance parameters,root-project/root/v6-32-06,code_comment,6
...,...,...,...,...
2992,Thanks it worked. ![screen shot 2018-11-20 at ...,scverse/scanpy/1.10.2,issue_comment,10
82,GPU Support for scispacy; Is GPU support avail...,allenai/scispacy/v0.5.5,issue,10
5447,"With that change, we also support cell_ranger ...",scverse/scanpy/1.10.2,issue_comment,10
9,"For the document level, we expand to contain n...",allenai/scispacy/v0.5.5,code_comment,10


In [14]:
df3 = df2[df2.nwords > 5]

In [15]:
df3.shape

(524909, 4)

## Save optimized data

In [18]:
split_dataset_by_repo_and_source(AbsDirPath.O2_KEYWORDS_MATCHING, df3, clean_before_saving=True, drop_columns_before_save=default_columns + ["nwords"])


C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\allenai.scispacy.v0.5.5.code_comment.parquet
Saved C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\allenai.scispacy.v0.5.5.code_comment.parquet
C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\allenai.scispacy.v0.5.5.docs.parquet
Saved C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\allenai.scispacy.v0.5.5.docs.parquet
C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\allenai.scispacy.v0.5.5.issue.parquet
Saved C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\allenai.scispacy.v0.5.5.issue.parquet
C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\allenai.scispacy.v0.5.5.issue_comment.parquet
Saved C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywor

In [20]:
from utilities.split_parquet import split_files_exceeding_max_limit

split_files_exceeding_max_limit(AbsDirPath.O2_KEYWORDS_MATCHING)

File size: C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\google.deepvariant.v1.6.1.issue.parquet | 5892704
Splitting file C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\google.deepvariant.v1.6.1.issue.parquet of size 5892704 into 3 chunks
File size: C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\google.deepvariant.v1.6.1.issue_comment.parquet | 6858367
Splitting file C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\google.deepvariant.v1.6.1.issue_comment.parquet of size 6858367 into 3 chunks
File size: C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\root-project.root.v6-32-06.code_comment.parquet | 40526251
Splitting file C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o2\root-project.root.v6-32-06.code_comment.parquet of size 40526251 into 16 chunks